In [80]:
import numpy as np
import json
def load_from_step(step):
    fn_npy = f'step_state_{step}.npy'
    fn_json = f'step_state_{step}.json'
    with open(fn_json, "r") as f:
        input_state = json.load(f)
    input_state['state'] = np.load(fn_npy)
    return input_state

load_from_step(10)

{'params': {'Name': 'adolescent#003',
  'i': 3,
  'x0_ 1': 0,
  'x0_ 2': 0,
  'x0_ 3': 0,
  'x0_ 4': 326.55338,
  'x0_ 5': 383.368339011,
  'x0_ 6': 3.48788064,
  'x0_ 7': 0,
  'x0_ 8': 101.28,
  'x0_ 9': 101.28,
  'x0_10': 5.27738007224,
  'x0_11': 80.7969837316,
  'x0_12': 81.3429092974,
  'x0_13': 326.55338,
  'BW': 44.791,
  'EGPb': 2.8199,
  'Gb': 145.9,
  'Ib': 101.28,
  'kabs': 0.046752,
  'kmax': 0.046773,
  'kmin': 0.0066262,
  'b': 0.56424,
  'd': 0.14423,
  'Vg': 2.2382,
  'Vi': 0.034438,
  'Ipb': 3.48788064,
  'Vmx': 0.060762,
  'Km0': 216.57,
  'k2': 0.080467,
  'k1': 0.10004,
  'p2u': 0.065784,
  'm1': 0.10962,
  'm5': 0.016546,
  'CL': 0.63961,
  'HEb': 0.6,
  'm2': 0.414654673732,
  'm4': 0.165861869493,
  'm30': 0.16443,
  'Ilb': 5.27738007224,
  'ki': 0.0094229,
  'kp2': 0.0060918,
  'kp3': 0.017571,
  'f': 0.9,
  'Gpb': 326.55338,
  'ke1': 0.0005,
  'ke2': 339,
  'Fsnc': 1,
  'Gtb': 383.368339011,
  'Vm0': 2.84798631515,
  'Rdb': 2.8199,
  'PCRb': 0.0193276216587,
  

In [19]:

with open('sim.json') as f:
    looped_sim = json.load(f)

def load_from_json(step):
    return next(k for k in looped_sim['states'] if k['t'] == float(step))
load_from_json(10)

{'params': {'Name': 'adolescent#003',
  'i': 3,
  'x0_ 1': 0,
  'x0_ 2': 0,
  'x0_ 3': 0,
  'x0_ 4': 326.55338,
  'x0_ 5': 383.368339011,
  'x0_ 6': 3.48788064,
  'x0_ 7': 0,
  'x0_ 8': 101.28,
  'x0_ 9': 101.28,
  'x0_10': 5.27738007224,
  'x0_11': 80.7969837316,
  'x0_12': 81.3429092974,
  'x0_13': 326.55338,
  'BW': 44.791,
  'EGPb': 2.8199,
  'Gb': 145.9,
  'Ib': 101.28,
  'kabs': 0.046752,
  'kmax': 0.046773,
  'kmin': 0.0066262,
  'b': 0.56424,
  'd': 0.14423,
  'Vg': 2.2382,
  'Vi': 0.034438,
  'Ipb': 3.48788064,
  'Vmx': 0.060762,
  'Km0': 216.57,
  'k2': 0.080467,
  'k1': 0.10004,
  'p2u': 0.065784,
  'm1': 0.10962,
  'm5': 0.016546,
  'CL': 0.63961,
  'HEb': 0.6,
  'm2': 0.414654673732,
  'm4': 0.165861869493,
  'm30': 0.16443,
  'Ilb': 5.27738007224,
  'ki': 0.0094229,
  'kp2': 0.0060918,
  'kp3': 0.017571,
  'f': 0.9,
  'Gpb': 326.55338,
  'ke1': 0.0005,
  'ke2': 339,
  'Fsnc': 1,
  'Gtb': 383.368339011,
  'Vm0': 2.84798631515,
  'Rdb': 2.8199,
  'PCRb': 0.0193276216587,
  

In [91]:
def compare_timestep(t):
    looped = load_from_json(t)
    stepwise = load_from_step(t)
    # print(looped['state'])
    # print(stepwise['state'])
    for v1, v2 in zip(looped['state'], stepwise['state']):
        print(f"{v1}\t\t\t{v2}\t\t\t\t{v1-v2}")
compare_timestep(1)

0.0			0.0				0.0
0.0			0.0				0.0
0.0			0.0				0.0
326.55337999999455			326.55337999999455				0.0
383.3683390110031			383.3683390110031				0.0
3.487880640000734			3.487880640000734				0.0
7.759212253562294e-13			7.759212253562294e-13				0.0
101.2800000000001			101.2800000000001				0.0
101.28			101.28				0.0
5.277380072238396			5.277380072238396				0.0
80.79698373160431			80.79698373160431				0.0
81.34290929739936			81.34290929739936				0.0
326.5533799999997			326.5533799999997				0.0


In [ ]:
def model(t, x, action, params, last_Qsto, last_foodtaken):
    dxdt = np.zeros(13)
    d = action.CHO * 1000  # g -> mg
    insulin = action.insulin * 6000 / params.BW  # U/min -> pmol/kg/min
    basal = params.u2ss * params.BW / 6000  # U/min
    # Glucose in the stomach
    qsto = x[0] + x[1]
    # NOTE: Dbar is in unit mg, hence last_foodtaken needs to be converted
    # from mg to g. See https://github.com/jxx123/simglucose/issues/41 for
    # details.
    Dbar = last_Qsto + last_foodtaken * 1000  # unit: mg

    # Stomach solid
    dxdt[0] = -params.kmax * x[0] + d

    if Dbar > 0:
        aa = 5 / (2 * Dbar * (1 - params.b))
        cc = 5 / (2 * Dbar * params.d)
        kgut = params.kmin + (params.kmax - params.kmin) / 2 * (
            np.tanh(aa * (qsto - params.b * Dbar)) - np.tanh(cc * (qsto - params.d * Dbar)) + 2
        )
    else:
        kgut = params.kmax

    # stomach liquid
    dxdt[1] = params.kmax * x[0] - x[1] * kgut

    # intestine
    dxdt[2] = kgut * x[1] - params.kabs * x[2]

    # Rate of appearance
    Rat = params.f * params.kabs * x[2] / params.BW
    # Glucose Production
    EGPt = params.kp1 - params.kp2 * x[3] - params.kp3 * x[8]
    # Glucose Utilization
    Uiit = params.Fsnc

    # renal excretion
    if x[3] > params.ke2:
        Et = params.ke1 * (x[3] - params.ke2)
    else:
        Et = 0

    # glucose kinetics
    # plus dextrose IV injection input u[2] if needed
    dxdt[3] = max(EGPt, 0) + Rat - Uiit - Et - params.k1 * x[3] + params.k2 * x[4]
    dxdt[3] = (x[3] >= 0) * dxdt[3]

    Vmt = params.Vm0 + params.Vmx * x[6]
    Kmt = params.Km0
    Uidt = Vmt * x[4] / (Kmt + x[4])
    dxdt[4] = -Uidt + params.k1 * x[3] - params.k2 * x[4]
    dxdt[4] = (x[4] >= 0) * dxdt[4]

    # insulin kinetics
    dxdt[5] = (
        -(params.m2 + params.m4) * x[5] + params.m1 * x[9] + params.ka1 * x[10] + params.ka2 * x[11]
    )  # plus insulin IV injection u[3] if needed
    It = x[5] / params.Vi
    dxdt[5] = (x[5] >= 0) * dxdt[5]

    # insulin action on glucose utilization
    dxdt[6] = -params.p2u * x[6] + params.p2u * (It - params.Ib)

    # insulin action on production
    dxdt[7] = -params.ki * (x[7] - It)

    dxdt[8] = -params.ki * (x[8] - x[7])

    # insulin in the liver (pmol/kg)
    dxdt[9] = -(params.m1 + params.m30) * x[9] + params.m2 * x[5]
    dxdt[9] = (x[9] >= 0) * dxdt[9]

    # subcutaneous insulin kinetics
    dxdt[10] = insulin - (params.ka1 + params.kd) * x[10]
    dxdt[10] = (x[10] >= 0) * dxdt[10]

    dxdt[11] = params.kd * x[10] - params.ka2 * x[11]
    dxdt[11] = (x[11] >= 0) * dxdt[11]

    # subcutaneous glucose
    dxdt[12] = -params.ksc * x[12] + params.ksc * x[3]
    dxdt[12] = (x[12] >= 0) * dxdt[12]

    if action.insulin > basal:
        print("t = {}, injecting insulin: {}".format(t, action.insulin))

    return dxdt


Name               adolescent#003
i                               3
x0_ 1                           0
x0_ 2                           0
x0_ 3                           0
                        ...      
u2ss                     1.446266
isc1ss                  80.796984
isc2ss                  81.342909
kp1                      6.588789
patient_history                 0
Length: 62, dtype: object
[0, 0.0107966168000268]
Name               adolescent#003
i                               3
x0_ 1                           0
x0_ 2                           0
x0_ 3                           0
                        ...      
u2ss                     1.446266
isc1ss                  80.796984
isc2ss                  81.342909
kp1                      6.588789
patient_history                 0
Length: 62, dtype: object
[1.74069809e+04 2.39671712e+03 1.85300236e+02 3.26734204e+02
 3.83173596e+02 4.19719502e+00 3.43411598e+00 1.01820018e+02
 1.01288607e+02 5.82319835e+00 1.88266682e+02 8.859703

In [133]:
from collections import namedtuple
import pandas as pd
from scipy.integrate import ode
step = 3
Action = namedtuple("Action", ['CHO', 'insulin'])
timestep_data = load_from_step(step)
# ODE solver
_odesolver = ode(model).set_integrator("dopri5")
_params = pd.Series(timestep_data['params'])
print(_params)
init_state = timestep_data['state']
subseq_state = load_from_step(step+1)
subseq_state_looped = load_from_json(step+1)
t0 = 0

print(subseq_state['last_action'])

_odesolver.set_initial_value(init_state, t0)
_last_Qsto = init_state[0] + init_state[1]
_last_foodtaken = 0
name = _params["Name"]
# print(_params)
action = Action(*subseq_state['last_action'])
sample_time = 1
_odesolver.set_f_params(
    action, _params, _last_Qsto, _last_foodtaken
)
if _odesolver.successful():
    res = _odesolver.integrate(_odesolver.t + sample_time)
else:
    print("ODE solver failed!!")
    raise
print(_odesolver.t)
print('current', _odesolver.y.tolist())
print('from stepwise', subseq_state['state'].tolist())
print('from looped', subseq_state_looped['state'])
for a,b,c in zip(_odesolver.y.tolist(), subseq_state['state'].tolist(), subseq_state_looped['state']):
    print(a,b,c,'\t\t\t\t\t\t\t',a-b, '\t\t',a-c)

Name               adolescent#003
i                               3
x0_ 1                           0
x0_ 2                           0
x0_ 3                           0
                        ...      
u2ss                     1.446266
isc1ss                  80.796984
isc2ss                  81.342909
kp1                      6.588789
patient_history                 0
Length: 62, dtype: object
[0, 0.0107966168000268]
1.0
current [0.0, 0.0, 0.0, 326.55337999998227, 383.3683390110083, 3.487880640001158, 6.167456777376776e-12, 101.28000000000097, 101.28, 5.277380072236096, 80.79698373161682, 81.34290929739787, 326.55337999999654]
from stepwise [0.0, 0.0, 0.0, 326.55337999998227, 383.3683390110083, 3.487880640001158, 6.167456777376776e-12, 101.28000000000097, 101.28, 5.277380072236096, 80.79698373161682, 81.34290929739787, 326.55337999999654]
from looped [0.0, 0.0, 0.0, 326.55337999998227, 383.36833901100823, 3.4878806400011584, 6.1688580075033954e-12, 101.28000000000095, 101.2800000000

In [ ]:
[
    16611.551453438253,
    3064.985436432285,
    301.194651117004,
    326.9125317398148,
    383.06461668121597,
    4.269265972483867,
    4.595814043141762,
    102.01819608670394,
    101.29450758916343,
    5.963612281100419,
    186.2543273668515,
    90.06500932524354,
    326.5960022519932,
]

326.5960022519932

In [134]:
init_state_ = [np.int64(0), np.int64(0), np.int64(0), np.float64(326.55338), np.float64(383.368339011), np.float64(3.48788064), np.int64(0), np.float64(101.28), np.float64(101.28), np.float64(5.27738007224), np.float64(80.7969837316), np.float64(81.3429092974), np.float64(326.55338)]
t0_ = 0
_odesolver.set_initial_value(init_state_, t0_)
print(_odesolver.t)
_last_Qsto = init_state[0] + init_state[1]
_last_foodtaken = 0
name = _params["Name"]
# print(_params)
action = Action(*subseq_state['last_action'])
print(action)
sample_time = 1
while _odesolver.t < step:
    _odesolver.set_f_params(
        action, _params, _last_Qsto, _last_foodtaken
    )
    if _odesolver.successful():
        res = _odesolver.integrate(_odesolver.t + sample_time)
    else:
        print("ODE solver failed!!")
        raise
print(_odesolver.t)
# print(init_state)
print('current', _odesolver.y.tolist())
print('from stepwise', subseq_state['state'].tolist())
print('from looped', subseq_state_looped['state'])
for a,b,c in zip(_odesolver.y.tolist(), subseq_state['state'].tolist(), subseq_state_looped['state']):
    print(a,b,c,'\t\t\t\t\t\t\t',a-b, '\t\t',a-c)

0
Action(CHO=0, insulin=0.0107966168000268)
3.0
current [0.0, 0.0, 0.0, 326.55337999998585, 383.3683390110071, 3.4878806400011464, 4.303739647289101e-12, 101.28000000000064, 101.28, 5.277380072236629, 80.79698373161273, 81.3429092973983, 326.5533799999978]
from stepwise [0.0, 0.0, 0.0, 326.55337999998227, 383.3683390110083, 3.487880640001158, 6.167456777376776e-12, 101.28000000000097, 101.28, 5.277380072236096, 80.79698373161682, 81.34290929739787, 326.55337999999654]
from looped [0.0, 0.0, 0.0, 326.55337999998227, 383.36833901100823, 3.4878806400011584, 6.1688580075033954e-12, 101.28000000000095, 101.28000000000002, 5.277380072236096, 80.79698373161682, 81.34290929739787, 326.5533799999964]
0.0 0.0 0.0 							 0.0 		 0.0
0.0 0.0 0.0 							 0.0 		 0.0
0.0 0.0 0.0 							 0.0 		 0.0
326.55337999998585 326.55337999998227 326.55337999998227 							 3.581135388230905e-12 		 3.581135388230905e-12
383.3683390110071 383.3683390110083 383.36833901100823 							 -1.1937117960769683e-12 		 -1.1